In [33]:
import os
from TwitterAPI import TwitterAPI
import tweepy
import sqlite3
import time
import argparse

**Task:** Load the values of access tokens and keys from environmental variables to python variables

In [34]:
consumer_key = os.environ.get('consumer_key')# CODE HERE
consumer_secret = os.environ.get('consumer_secret') # CODE HERE
access_token = os.environ.get('access_token') # CODE HERE
access_token_secret =os.environ.get('access_secret') # CODE HERE
bearer_token = os.environ.get('bearer_token')

In [36]:
#create the sqlite3 database to store Data from tweets
conn = sqlite3.connect("../data/tweets.db")
print("DB created!")
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS tweets (username TEXT, tweet TEXT)")
print("Table created")

DB created!
Table created


In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [40]:
# Checking the type of api object
api

In [147]:
import requests
import json

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


def get_rules():
    #return all current rules
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    #delete all current rules
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(delete):
    #set new rules
    twt_rules = [
        {"value": "Python lang:en", "tag": "Python"}
    ]
    payload = {"add": twt_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    

def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            print(json.dumps(json_response, indent=4, sort_keys=True))


def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    get_stream(set)


if __name__ == "__main__":
    main()

Response(data=[StreamRule(value='Python lang:en', tag='Python', id='1601819466991558656')], includes={}, errors=[], meta={'sent': '2022-12-11T06:27:30.897Z', 'result_count': 1})

In [ ]:
class TweetStream(tweepy.StreamingClient):
    """
    This class will be activted to listen new tweets (real-times) and
    store into DB created
    """
    def on_connect(self):
        #let us know that we have successfully connected to the twitter API
        print("Connected!")
    
        
    def on_tweet(self, tweet):
        #get real-time tweets
        if tweet.referenced_tweets != None:
            print(tweet.text) 
            
stream = TweetStream(bearer_token)

#..Params for functions
# Apply rules
#return current rules
#stream.get_rules()
#add new rules
stream.add_rules(tweepy.StreamRule(value="Python lang:en", tag="Python"))
stream.add_rules(tweepy.StreamRule(value="Data Science has:mentions lang:en", tag="Data Science"))

#return streaming tweets
stream.filter(tweet_fields=["referenced_tweets"])

Connected!
RT @TruthHurtsDemz: @HespeChristina @DanteOpt @XRebellionUK Climate Change “science”:

✅ Untestable thesis
✅ Hypocritical evangelists
✅ Dis…
RT @jnalanko: I asked ChatGPT to write a FASTA parser. It returned a fully working Python implementation with comments and everything. I as…
RT @swapnakpanda: Best YouTube channels for Programming:

❯ C ➟ Telusko
❯ C++ ➟ thenewboston
❯ Java ➟ Derek Banas
❯ C# ➟ dotnet
❯ PHP ➟ PHP…
RT @jblefevre60: 💥Can't wait to learn more about #QuantumComputing at #CES2023!

#AI #MachineLearning #Cloud #Python #Coding #100DaysOfCode…
RT @pendeviejo95: LMAOOO acting like social science/qual workers don’t collect data
RT @MemesDnd: Monty Python and the Curse of the Nat 1.
⚔️⚔️
credit to: jrex-42
#dnd  #rpg  #dndcharacter  #charactersheet  #d20  #follow  #…
RT @PythonPr: Build a Simple Blockchain in Python
#python #blockchain 
https://t.co/ktjsTgIBI8
RT @MemesDnd: Monty Python and the Curse of the Nat 1.
⚔️⚔️
credit to: jrex-42
#dnd  #rpg  #dndcharacter 

In [150]:
class TweetStream(tweepy.StreamingClient):
    """
    This class will be activted to listen new tweets (real-times) and
    store into DB created
    """
    def on_connect(self):
        #let us know that we have successfully connected to the twitter API
        print("Connected!")
    
    def on_includes(self, includes):
        #collect user details and adds to tweet database
        self.new_tweet["username"] = includes["users"][0].username
        print(self.new_tweet)
        #insert tweets in bd
        cursor.execute("INSERT INTO tweets VALUES (?,?)",
                       (
                           self.new_tweet['username'],
                           self.new_tweet['tweet'],
                       ),
                      )
        conn.commit()
        print(self.new_tweet)
        print("real-time tweet added to sqlite3 tweet database!")
        print("-" * 30)
        
    def on_tweet(self, tweet):
        if tweet.referenced_tweets == None:
            self.new_tweet['tweet'] = tweet.text
        print(tweet.text)
        
stream = TweetStream(bearer_token)
#..Params for functions

#locations to store tweets
LOCATION = '../data/'
#threshold to stop streaming
THRESHOLD = 10000

# Apply rules
#add new rules
stream.add_rules(tweepy.StreamRule(value="Python lang:en", tag="Python"))
stream.add_rules(tweepy.StreamRule(value="Data Science has:mentions", tag="Data Science"))
#return recently added rules
stream.get_rules()
#return streaming tweets
stream.filter(tweet_fields=["referenced_tweets"])

Connected!
RT @swapnakpanda: Best YouTube channels for Programming:

❯ C ➟ Telusko
❯ C++ ➟ thenewboston
❯ Java ➟ Derek Banas
❯ C# ➟ dotnet
❯ PHP ➟ PHP…
RT @_ijamy: If you are in need of someone to do writing for you in the field of software development you can DM me.
I have specialised in t…


Stream encountered an exception
Traceback (most recent call last):
  File "/opt/miniconda3/envs/lighthouse/lib/python3.9/site-packages/tweepy/streaming.py", line 94, in _connect
    self.on_data(line)
  File "/opt/miniconda3/envs/lighthouse/lib/python3.9/site-packages/tweepy/streaming.py", line 936, in on_data
    self.on_tweet(tweet)
  File "/var/folders/rn/ljpx9hjx4yvftbhwbld6fwjc0000gn/T/ipykernel_1062/294830648.py", line 29, in on_tweet
    self.new_tweet['tweet'] = tweet.text
AttributeError: 'TweetStream' object has no attribute 'new_tweet'


In [27]:
## FOLLOWING FUNCTION WILL COLLECT REAL-TIME TWEETS IN OUR COMPUTER
def main():
    with open('output.txt', 'a') as f:
        # api.GetStreamFilter will return a generator that yields one status
        # message (i.e., Tweet) at a time as a JSON dictionary.
        for line in api.GetStreamFilter(track=FILTER, languages=LANGUAGES):
            f.write(json.dumps(line))
            f.write('\n')

In [28]:
# Execute function
main()

AttributeError: 'API' object has no attribute 'GetStreamFilter'

**Task:** Edit function `main` so it can store tweets anywhere (location specified as parameter). The FILTER and LANGUAGES should be parameters as well. Test it with different values and languages.

**Task:** Create File `stream_tweets.py` that can be executed from the Terminal by exporting the code from this notebook.

**Task:** Start storing tweets which contain either happy smiley (`:)`) or sad smiley (`:(`) in their text. We will use this dataset during the NLP section. It's good to let the script running for at least 2-3 hours to collect enough data for future modeling.

> #### Note
> the main function runs as an ongoing process and won;t stop until you stop it!